In [23]:
import numpy as np
import pandas as pd

import poissonlearning as pl
import graphlearning as gl

import storage

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
dataset = "fashionmnist"
dataset_metric = "vae"
experiments = storage.load_results(name="real_data", folder="../results")

In [25]:
p_all = list(set((ex["p"] for ex in experiments if ex["dataset"] == dataset and ex["dataset_metric"] == dataset_metric)))
results_table = pd.DataFrame()
for p in p_all:
    labels_per_class_all = list(set((ex["labels_per_class"][0] for ex in experiments if np.isclose(ex["p"], p) and ex["dataset"] == dataset and ex["dataset_metric"] == dataset_metric)))
    results_p = pd.Series(index=labels_per_class_all, name=p, dtype="object")
    for num_labels in labels_per_class_all:
        selected_experiments = list(
            filter(
                lambda x: 
                    np.isclose(x["p"], p) 
                    and np.all(np.isclose(x["labels_per_class"], num_labels)),
                experiments
            )
        )
        def _compute_accuracy(experiment):
            prob = experiment["solution"].drop(columns=["x", "y", "true_labels"]).to_numpy()
            scores = prob - np.min(prob)
            scores = scores / np.max(scores)

            # Check if scores are similarity or distance
            pred_labels = np.argmax(scores, axis=1)
            accuracy = gl.ssl.ssl_accuracy(experiment["solution"]["true_labels"], pred_labels, sum(experiment["labels_per_class"]))
            return accuracy
        accuracy = [_compute_accuracy(ex) for ex in selected_experiments]
        accuracy_mean = np.mean(accuracy)
        accuracy_std = np.std(accuracy)
        results_p[num_labels] = f"{accuracy_mean:.3f} ({accuracy_std:.2f})"

    def _extend_results(results_table, new_entries):
        new_index = np.union1d(results_table.index, new_entries.index)
        results_table = results_table.reindex(new_index)
        new_entries = new_entries.reindex(new_index)
        results_table[new_entries.name] = new_entries
        return results_table

    results_table = _extend_results(results_table, results_p)
results_table.T

,5
2.0,68.556 (1.38)
3.0,65.545 (0.86)
4.0,63.485 (0.58)
